In [1]:
import elasticsearch
from datetime import datetime

In [15]:
country="chile"
from_="2021-02-01"
to_="2021-02-28"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

keyword=""
simple_keyword=False

In [16]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [17]:
#NO MODIFICAR

es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

In [18]:
#NO MODIFICAR
match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

query = { 
    "bool": { 
      #"must": [
      #  {match: { "text":keyword}}

      #],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [19]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 6444 noticias encontradas...


In [20]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    
    df = df.append(new_row, ignore_index=True)

df

,id_news,country,media_outlet,url,title,text,date
0,21887877.0,chile,radiosago,https://www.radiosago.cl/15-equipos-de-vacunac...,15 equipos de vacunación dispuso el municipio ...,Hasta la Escuela Rural de Chamiza llegó el Se...,2021-02-10
1,21887907.0,chile,radiosago,https://www.radiosago.cl/plantas-de-revision-t...,Plantas de Revisión Técnica de Puerto Montt y ...,Con el objetivo de disminuir la espera y evit...,2021-02-03
2,21917228.0,chile,radiosago,https://www.radiosago.cl/comenzo-el-ingreso-de...,Comenzó el ingreso de oferta de recursos de la...,A través de esta iniciativa en el sitio web de...,2021-02-20
3,21917905.0,chile,radiosago,https://www.radiosago.cl/sernapesca-actualizo-...,Sernapesca actualizó cierre de áreas por marea...,El Servicio Nacional de Pesca y Acuicultura in...,2021-02-06
4,21917424.0,chile,radiosago,https://www.radiosago.cl/encapuchados-protagon...,Encapuchados protagonizaron un ataque incendia...,"Pasadas las 04:00 horas de este viernes, un gr...",2021-02-19
...,...,...,...,...,...,...,...
6439,47507259.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Atrapan a 4 estudiantes por ataque a liceo por...,Con una salida alternativa quedaron los cuatro...,2021-02-08
6440,47507260.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Queman carpa fuera de la ex Feria Yumbel,Preocupados se encuentran los vecinos aledaños...,2021-02-08
6441,47507359.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Billie Eilish anuncia que ya tiene listo su pr...,La cantante Bilie Eilish reveló en conversació...,2021-02-27
6442,47507360.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,"HBO compra serie chilena ""Isabel"" para emitirl...","La serie ""Isabel"", basada en la historia íntim...",2021-02-27


In [21]:
df['url'].duplicated().any()

True

In [22]:
df = df.drop_duplicates(subset='url', keep='first')

In [23]:
nombre_archivo="01-02-21.csv"
df.to_csv(nombre_archivo)